# Python to C++ Code Translator using LLMs

This notebook translates Python code to compilable C++ using GPT, Gemini, or Claude.

## Features:
- 🤖 Multiple LLM support (GPT, Gemini, Claude)
- ✅ Automatic compilation testing with g++
- 🔄 Comparison mode to test all LLMs
- 💬 Interactive translation mode

## Step 1: Install Required Packages

Run this cell first to install all dependencies:

In [ ]:
!uv add openai anthropic python-dotenv google-generativeai
#!pip install openai anthropic python-dotenv google-generativeai

## Step 2: Import Libraries

In [ ]:
import os
import subprocess
import tempfile
from pathlib import Path
from dotenv import load_dotenv
import openai
from anthropic import Anthropic
import google.generativeai as genai

## Step 3: Load API Keys

Make sure you have a `.env` file with:
```
OPENAI_API_KEY=your_key_here
GEMINI_API_KEY=your_key_here
ANTHROPIC_API_KEY=your_key_here
```

In [ ]:
# Load API keys from .env file
load_dotenv()

# Initialize API clients
openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
anthropic_client = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

print("✓ API keys loaded successfully")

## Step 4: Define System Prompt

In [ ]:
SYSTEM_PROMPT = """You are an expert programmer that translates Python code to C++.
Translate the given Python code to efficient, compilable C++ code.

Requirements:
- The C++ code must compile without errors
- Include all necessary headers
- Use modern C++ (C++11 or later) features where appropriate
- Add proper error handling
- Maintain the same functionality as the Python code
- Include a main() function if the Python code has executable statements

Only return the C++ code, no explanations unless there are important notes about compilation."""

## Step 5: LLM Translation Functions

In [ ]:
def translate_with_gpt(python_code, model="gpt-4o"):
    """Translate Python to C++ using OpenAI's GPT models"""
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"Translate this Python code to C++:\n\n{python_code}"}
            ],
            temperature=0.2
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error with GPT: {str(e)}"

def translate_with_gemini(python_code, model="gemini-2.0-flash-exp"):
    """Translate Python to C++ using Google's Gemini"""
    try:
        model_instance = genai.GenerativeModel(model)
        prompt = f"{SYSTEM_PROMPT}\n\nTranslate this Python code to C++:\n\n{python_code}"
        response = model_instance.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error with Gemini: {str(e)}"

def translate_with_claude(python_code, model="claude-sonnet-4-20250514"):
    """Translate Python to C++ using Anthropic's Claude"""
    try:
        response = anthropic_client.messages.create(
            model=model,
            max_tokens=4096,
            temperature=0.2,
            system=SYSTEM_PROMPT,
            messages=[
                {"role": "user", "content": f"Translate this Python code to C++:\n\n{python_code}"}
            ]
        )
        return response.content[0].text
    except Exception as e:
        return f"Error with Claude: {str(e)}"

## Step 6: Main Translation Function

In [ ]:
def translate_python_to_cpp(python_code, llm="gpt", model=None):
    """
    Translate Python code to C++ using specified LLM
    
    Args:
        python_code (str): Python code to translate
        llm (str): LLM to use ('gpt', 'gemini', or 'claude')
        model (str): Specific model version (optional)
    
    Returns:
        str: Translated C++ code
    """
    print(f"🔄 Translating with {llm.upper()}...")
    
    if llm.lower() == "gpt":
        model = model or "gpt-4o"
        cpp_code = translate_with_gpt(python_code, model)
    elif llm.lower() == "gemini":
        model = model or "gemini-2.0-flash-exp"
        cpp_code = translate_with_gemini(python_code, model)
    elif llm.lower() == "claude":
        model = model or "claude-sonnet-4-20250514"
        cpp_code = translate_with_claude(python_code, model)
    else:
        return "Error: Invalid LLM. Choose 'gpt', 'gemini', or 'claude'"
    
    return cpp_code

## Step 7: Compilation Testing Functions

In [ ]:
def extract_cpp_code(text):
    """Extract C++ code from markdown code blocks if present"""
    if "```cpp" in text:
        start = text.find("```cpp") + 6
        end = text.find("```", start)
        return text[start:end].strip()
    elif "```c++" in text:
        start = text.find("```c++") + 6
        end = text.find("```", start)
        return text[start:end].strip()
    elif "```" in text:
        start = text.find("```") + 3
        end = text.find("```", start)
        return text[start:end].strip()
    return text.strip()

def compile_cpp_code(cpp_code, output_name="translated_program"):
    """
    Compile C++ code and return compilation status
    
    Args:
        cpp_code (str): C++ code to compile
        output_name (str): Name of output executable
    
    Returns:
        dict: Compilation result with status and messages
    """
    # Extract code from markdown if present
    cpp_code = extract_cpp_code(cpp_code)
    
    # Create temporary directory
    with tempfile.TemporaryDirectory() as tmpdir:
        cpp_file = Path(tmpdir) / "program.cpp"
        exe_file = Path(tmpdir) / output_name
        
        # Write C++ code to file
        with open(cpp_file, 'w') as f:
            f.write(cpp_code)
        
        # Try to compile
        try:
            result = subprocess.run(
                ['g++', '-std=c++17', str(cpp_file), '-o', str(exe_file)],
                capture_output=True,
                text=True,
                timeout=10
            )
            
            if result.returncode == 0:
                return {
                    'success': True,
                    'message': '✓ Compilation successful!',
                    'executable': str(exe_file),
                    'stdout': result.stdout,
                    'stderr': result.stderr
                }
            else:
                return {
                    'success': False,
                    'message': '✗ Compilation failed',
                    'stdout': result.stdout,
                    'stderr': result.stderr
                }
        except subprocess.TimeoutExpired:
            return {
                'success': False,
                'message': '✗ Compilation timed out'
            }
        except FileNotFoundError:
            return {
                'success': False,
                'message': '✗ g++ compiler not found. Please install g++ to compile C++ code.'
            }
        except Exception as e:
            return {
                'success': False,
                'message': f'✗ Compilation error: {str(e)}'
            }

## Step 8: Complete Pipeline

In [ ]:
def translate_and_compile(python_code, llm="gpt", model=None, verbose=True):
    """
    Translate Python to C++ and attempt compilation
    
    Args:
        python_code (str): Python code to translate
        llm (str): LLM to use
        model (str): Specific model version
        verbose (bool): Print detailed output
    
    Returns:
        dict: Results including translated code and compilation status
    """
    # Translate
    cpp_code = translate_python_to_cpp(python_code, llm, model)
    
    if verbose:
        print("\n" + "="*60)
        print("TRANSLATED C++ CODE:")
        print("="*60)
        print(cpp_code)
        print("="*60 + "\n")
    
    # Compile
    print("🔨 Attempting to compile...")
    compilation_result = compile_cpp_code(cpp_code)
    
    if verbose:
        print(compilation_result['message'])
        if not compilation_result['success'] and 'stderr' in compilation_result:
            print("\nCompilation errors:")
            print(compilation_result['stderr'])
    
    return {
        'cpp_code': cpp_code,
        'compilation': compilation_result
    }

## Example 1: Factorial Function

In [ ]:
python_code_1 = """
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)

# Test the function
print(factorial(5))
"""

print("Example 1: Factorial Function")
print("="*60)
result1 = translate_and_compile(python_code_1, llm="gpt")

## Example 2: Sum of Squares

In [ ]:
python_code_2 = """
def sum_of_squares(numbers):
    return sum(x**2 for x in numbers)

numbers = [1, 2, 3, 4, 5]
result = sum_of_squares(numbers)
print(f"Sum of squares: {result}")
"""

print("Example 2: Sum of Squares")
print("="*60)
result2 = translate_and_compile(python_code_2, llm="claude")

## Example 3: Fibonacci with Gemini

In [ ]:
python_code_3 = """
def fibonacci(n):
    if n <= 1:
        return n
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

print(f"Fibonacci(10) = {fibonacci(10)}")
"""

print("Example 3: Fibonacci with Gemini")
print("="*60)
result3 = translate_and_compile(python_code_3, llm="gemini")

## Compare All LLMs

In [ ]:
def compare_llms(python_code):
    """Compare all three LLMs on the same Python code"""
    llms = ["gpt", "gemini", "claude"]
    results = {}
    
    for llm in llms:
        print(f"\n{'='*60}")
        print(f"Testing with {llm.upper()}")
        print('='*60)
        results[llm] = translate_and_compile(python_code, llm=llm, verbose=False)
        print(results[llm]['compilation']['message'])
    
    return results

In [ ]:
# Test code for comparison
python_code_compare = """
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = [x for x in range(2, 20) if is_prime(x)]
print(f"Primes under 20: {primes}")
"""

print("COMPARING ALL LLMs")
print("="*60)
comparison_results = compare_llms(python_code_compare)

## Interactive Translation Mode

Use this cell to translate your own Python code interactively:

In [ ]:
# Your custom Python code here
your_python_code = """
# Paste your Python code here
def hello_world():
    print("Hello, World!")

hello_world()
"""

# Choose your LLM: "gpt", "gemini", or "claude"
chosen_llm = "gpt"

result = translate_and_compile(your_python_code, llm=chosen_llm)

## Summary

You now have a complete Python to C++ translator! 

### Main Functions:
- `translate_python_to_cpp(code, llm, model)` - Translate only
- `translate_and_compile(code, llm, model)` - Translate and compile
- `compare_llms(code)` - Compare all three LLMs

### Supported LLMs:
- **gpt** - OpenAI GPT-4o
- **gemini** - Google Gemini 2.0 Flash
- **claude** - Anthropic Claude Sonnet 4

Happy translating! 🚀